# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up



Les bases de données multidimensionnelles considèrent chaque attribut d’une donnée comme une dimension « séparée ». Le logiciel peut ensuite localiser l’intersection des dimensions et les afficher. Il est ainsi possible d’analyser et de comparer les données de différentes façons. Les attributs peuvent aussi être séparés en plusieurs sous-attributs. Les bases de données multi-dimensionnelles s’opposent aux bases de données relationnelles à deux dimensions.

In [2]:
# Do all imports and installs here


import os
import sys
import boto3

import datetime
import numpy as np
import pandas as pd

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 2)


In [5]:
# Read in the data here
!ls -1FSash ./dataset
#!ls -tRFh ./dataset/
path = './dataset/'

total 1,2G
510M airports_us.csv
509M GlobalLandTemperaturesByCity.csv
205M WDIData.csv
5,8M airport-codes_csv.csv
248K us-cities-demographics.csv
144K immigration_data_sample.csv
 36K I94_SAS_Labels_Descriptions.SAS
4,0K ./
4,0K ../
4,0K 20-years-us-university-dataset/
4,0K airline-delay-and-cancellation-data-2009-2018/
4,0K education-statistics/
4,0K sas_data/


# Step 1: Scope the Project and Gather Data

#### Scope TODO
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data TODO
Describe the data sets you're using. Where did it come from? What type of information is included? 

# Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

### Immigration data

 * U.S. immigration officers
 * I-94 Form (Arrival/Departure Record) to foreign visitors 
 * Explanation about the Visitor Arrivals Program (I-94 Form) [here](https://travel.trade.gov/research/programs/i94/description.asp)
 * data about arrival and why and during
 * dataset = 1 file per month
 * here: april 2026

TODO
* immigration_data_sample
    * revoir l'orignine du fichier. chercher data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat dans udacity workspace
    (pd.read_sas(immigration_fname, 'sas7bdat', encoding="ISO-8859-1")
    ==> 1000 rows fourni par udac, beaucoup plus dans 1 seul mois
    * DONE: faire un dictionnaire, recuperer les colonnes. 
* chercher les valeurs manquantes, les valeurs dupliquees, data cleanning

In [156]:
nRowsRead = 1000 # change and set to None for the whole data
description = "dataset immigration provide by Udacity"
name = "df_immigration"
file = "immigration_data_sample.csv"
n_df = pd.read_csv(path+file, nrows = nRowsRead)
nRow, nCol = n_df.shape
print("There are {} rows and {} columns in {}.".format(nRow, nCol, description))
df_immigration = n_df

n_df.head(3)
print(n_df.info())
#dic_6 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}

There are 1000 rows and 29 columns in dataset immigration provide by Udacity.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1000 non-null   int64  
 1   cicid       1000 non-null   float64
 2   i94yr       1000 non-null   float64
 3   i94mon      1000 non-null   float64
 4   i94cit      1000 non-null   float64
 5   i94res      1000 non-null   float64
 6   i94port     1000 non-null   object 
 7   arrdate     1000 non-null   float64
 8   i94mode     1000 non-null   float64
 9   i94addr     941 non-null    object 
 10  depdate     951 non-null    float64
 11  i94bir      1000 non-null   float64
 12  i94visa     1000 non-null   float64
 13  count       1000 non-null   float64
 14  dtadfile    1000 non-null   int64  
 15  visapost    382 non-null    object 
 16  occup       4 non-null      object 
 17  entdepa     1000 non-null   obje

##### Data Dictionary

Column Name | Description | Example | Type
-|-|-|-|
**cicid**|     ID uniq per record in the dataset | 4.08e+06 | float64
**i94yr**|     4 digit year  | 2016.0 | float64
**i94mon**|    Numeric month |  4.0 | float64
**i94cit**|     3 digit code of source city for immigration (Born country) | 209.0 | float64
**i94res**|    3 digit code of source country for immigration (Residence country) | 209.0 | float64
**i94port**|   Port addmitted through | HHW | object
**arrdate**|   Arrival date in the USA | 20566.0 | float64
**i94mode**|   Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported) | 1.0 | float
**i94addr**|   State of arrival | HI | object
**depdate**|   Departure date | 20573.0 | float
**i94bir**|    Age in years | 61.0 | float
**i94visa**|   Visa Code - 1 = Business / 2 = Pleasure / 3 = Student |2.0 | float
**count**|     Used for summary statistics | 1.0 | float
**dtadfile**|  Character Date Field |20160422| int 64 
**visapost**|  Department of State where where Visa was issued | | object
**occup**|     Occupation that will be performed in U.S. || object
**entdepa**|   Arrival Flag - Whether admitted or paroled into the US |G| object
**entdepd**|   Departure Flag. Whether departed, lost visa, or deceased |O|  object
**entdepu**|   Update Flag - Either apprehended, overstayed, adjusted to perm residence || **float64
**matflag**|   Match flag |M|  object
**biryear**|   4 digit year of birth |1955.0| float64
**dtaddto**|   Character date field to when admitted in the US |07202016| object
**gender**|    Gender|M| object
**insnum**|    INS number || float64
**airline**|   Airline used to arrive in U.S.|JL| object
**admnum**|    Admission number, should be unique and not nullable |5.66e+10| float
**fltno**|     Flight number of Airline used to arrive in U.S. |00782| object
**visatype**|  Class of admission legally admitting the non-immigrant to temporarily stay in U.S.|WT|object

###### Missing and Duplicate
* column `entdepu` has 1000/1000 missing values # Update Flag - Either apprehended, overstayed, adjusted to perm residence
* column `occup` has 996/1000 missing value # Occupation that will be performed in U.S.
* column `visapost` has 728/1000 missing value # Department of State where where Visa was issued
* column `insnum` has 965/1000 missing value # INS number
* somme value missing:
    * i94addr # US State of arrival 51 unique value
    * depdate # Departure date
    * entdepd # Departure Flag. Whether departed, lost visa, or deceased
    * matflag # Match flag 
    * gender  # Gender
    * airline # Airline used to arrive in U.S
    * fltno   # Flight number of Airline used to arrive in U.S


In [160]:
# number of unique value in columns
n_df.nunique()

Unnamed: 0    1000
cicid         1000
i94yr            1
i94mon           1
i94cit          88
i94res          91
i94port         70
arrdate         30
i94mode          4
i94addr         51
depdate        109
i94bir          85
i94visa          3
count            1
dtadfile        39
visapost        97
occup            3
entdepa          9
entdepd         10
entdepu          0
matflag          1
biryear         85
dtaddto         99
gender           3
insnum          29
airline        101
admnum        1000
fltno          502
visatype        10
dtype: int64

In [163]:
# check null value
cols = [col for col in n_df.columns if n_df[col].isnull().any()]
df_miss = n_df[cols]
print(df_miss.nunique())
df_miss.head()

i94addr      51
depdate     109
visapost     97
occup         3
entdepd      10
entdepu       0
matflag       1
gender        3
insnum       29
airline     101
fltno       502
dtype: int64


,i94addr,depdate,visapost,occup,entdepd,entdepu,matflag,gender,insnum,airline,fltno
0,HI,20573.0,NaN,NaN,O,NaN,M,F,NaN,JL,00782
1,TX,20568.0,MTR,NaN,R,NaN,M,M,NaN,*GA,XBLNG
2,FL,20571.0,NaN,NaN,O,NaN,M,M,NaN,LH,00464
3,CA,20581.0,DOH,NaN,O,NaN,M,M,NaN,QR,00739
4,NY,20553.0,NaN,NaN,K,NaN,M,F,NaN,NaN,LAND


In [159]:
df_dup = n_df[n_df.duplicated(keep=False)].sort_values("cicid")
df_dup

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype


**c/c: we have an ID for each record, `admnum` should be none null as `fltno `, `i94addr` for the analytic questions**

### Global Land Temperature Data

* data from kaggle
* who made it?
* this data date about city temperature around the world

TODO:
* ajouter les codes des pays en rapport avec le dataset immigration?
* reseigner qui a confectionner les dataset
* refaire avec spark, big data set

In [182]:
# download from kaggle the GlobalLandTemperaturesByCity.csv KAGGLE/UDACITY
nRowsRead = None # change and set to None for the whole data
description = "download from kaggle the GlobalLandTemperaturesByCity.csv KAGGLE/UDACITY"
name = "df_temp"
file = "GlobalLandTemperaturesByCity.csv"
n_df = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_temp = n_df

print(n_df.info())
#n_df.head(3)

#dic_2 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}


There are 8599212 rows and 7 columns in DataFrame df_temp.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   dt                             object 
 1   AverageTemperature             float64
 2   AverageTemperatureUncertainty  float64
 3   City                           object 
 4   Country                        object 
 5   Latitude                       object 
 6   Longitude                      object 
dtypes: float64(2), object(5)
memory usage: 459.2+ MB
None


##### Data Dictionary

Column Name | Description | Example | Type
-|-|-|-|
**dt**|Date format YYYY-MM-DD| 1743-11-01| object
**AverageTemperature**|Average Temperature for the city to th date dt|6.07|float64
**AverageTemperatureUncertainty**| Average Temperature Uncertainty | 1.74 |float64
**City**| City name| Århus| object
**Country**| Country name | Denmark | object
**Latitude**| Latitude| 57.05N | object
**Longitude** | Longitude | 10.33E |object


##### Missing and Duplicate
* no duplicate for the whole Temperature dataset
* 364130 none value for AverageTemperature, so remove this rows if we need temp
* begin in 1743 and finis in 2013 so agregation by country and year

In [176]:
df_dup = n_df[n_df.duplicated(keep=False)].sort_values("dt")
df_dup

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude


In [170]:
#check number of unique value in columns
n_df.nunique()

dt                                 3239
AverageTemperature               111994
AverageTemperatureUncertainty     10902
City                               3448
Country                             159
Latitude                             73
Longitude                          1227
dtype: int64

In [184]:
cols = [col for col in n_df.columns if n_df[col].isnull().any()]
df_miss = n_df[cols]
df_miss.isna().sum()

AverageTemperature               364130
AverageTemperatureUncertainty    364130
dtype: int64

In [172]:
n_df.sort_values(["dt"], ascending=True, inplace=True)
print(n_df["dt"].min())
print(n_df["dt"].max())

1743-11-01
2013-09-01


In [173]:
n_df["Country"].unique()

array(['Denmark', 'Bulgaria', 'Poland', 'Romania', 'United Kingdom',
       'Czech Republic', 'Montenegro', 'Russia', 'United States',
       'Ukraine', 'Portugal', 'Germany', 'Italy', 'Serbia', 'Spain',
       'Lithuania', 'France', 'Belarus', 'Canada', 'Netherlands',
       'Latvia', 'Croatia', 'Greece', 'Belgium', 'Bosnia And Herzegovina',
       'Hungary', 'Norway', 'Finland', 'Sweden', 'Austria', 'Turkey',
       'Tunisia', 'Macedonia', 'Estonia', 'Moldova', 'Albania', 'Ireland',
       'Switzerland', 'Slovakia', 'Slovenia', 'Iceland', 'Algeria',
       'Libya', 'Morocco', 'Bahamas', 'Kazakhstan', 'Armenia', 'Georgia',
       'Madagascar', 'Mozambique', 'Mauritius', 'Reunion', 'Egypt',
       'Syria', 'Cyprus', 'Lebanon', 'India', 'Pakistan', 'Bangladesh',
       'Burma', 'Sri Lanka', 'Nepal', 'Indonesia', 'Israel', 'Iran',
       'Azerbaijan', 'Iraq', 'Jordan', 'Turkmenistan', 'Uzbekistan',
       'China', 'Laos', 'Thailand', 'Saudi Arabia', 'Mongolia', 'Mexico',
       'Cuba', '

In [174]:
global_temp = df_temp.groupby(['Country']) \
                            .agg({"AverageTemperature": "mean", 
                                  "Latitude": "first", 
                                  "Longitude": "first"}).reset_index()
                            
global_temp.sort_values(["AverageTemperature"], ascending=True, inplace=True)
global_temp.tail()

,Country,AverageTemperature,Latitude,Longitude
89,Mali,27.59,15.27N,4.17W
20,Burkina Faso,27.82,12.05N,1.64W
133,Sudan,28.07,8.84N,31.62E
103,Niger,28.15,13.66N,2.48E
40,Djibouti,29.15,12.05N,42.74E


**c/c: Temperature from 1743 to 2013, useful if we want look for raison of immigration. People have needed bad weather conditions for a long time to leave their country**.

### Airports Data

* data from ? see udacity
* code used in passenger reservation, ticket and baggage-handling
* definition IATA
* definition ISO
* GPS code of airport

TODO:
    * remplir les points precedents
    * 

In [194]:
# airport-codes_csv UDACITY
nRowsRead = None # change and set to None for the whole data
description      = "airport-codes_csv provide by UDACITY"
name             = "df_airport_world"
file             = "airport-codes_csv.csv"

n_df             = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol       = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_airport_world = n_df

#dic_4 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
print(n_df.info())
n_df.head(10)

There are 55075 rows and 12 columns in DataFrame df_airport_world.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ident         55075 non-null  object 
 1   type          55075 non-null  object 
 2   name          55075 non-null  object 
 3   elevation_ft  48069 non-null  float64
 4   continent     27356 non-null  object 
 5   iso_country   54828 non-null  object 
 6   iso_region    55075 non-null  object 
 7   municipality  49399 non-null  object 
 8   gps_code      41030 non-null  object 
 9   iata_code     9189 non-null   object 
 10  local_code    28686 non-null  object 
 11  coordinates   55075 non-null  object 
dtypes: float64(1), object(11)
memory usage: 5.0+ MB
None


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"
5,00AS,small_airport,Fulton Airport,1100.0,NaN,US,US-OK,Alex,00AS,NaN,00AS,"-97.8180194, 34.9428028"
6,00AZ,small_airport,Cordes Airport,3810.0,NaN,US,US-AZ,Cordes,00AZ,NaN,00AZ,"-112.16500091552734, 34.305599212646484"
7,00CA,small_airport,Goldstone /Gts/ Airport,3038.0,NaN,US,US-CA,Barstow,00CA,NaN,00CA,"-116.888000488, 35.350498199499995"
8,00CL,small_airport,Williams Ag Airport,87.0,NaN,US,US-CA,Biggs,00CL,NaN,00CL,"-121.763427, 39.427188"
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350.0,NaN,US,US-CA,Pine Valley,00CN,NaN,00CN,"-116.4597417, 32.7273736"


### Data dictionary

Column Name | Description | Example | Type
-|-|-|-|
**ident**| Unique identifier| 00AK| object 
**type**| Type of airport | small_airport |object
**name**| Name of the airport | Lowell Field | object
**elevation_ft**| Altitude of the airport |11.0 |float
**continent**| Continent | | object
**iso_country**| ISO code of airport country |US| object
**iso_region**| ISO code of the region airport | US-KS|object
**municipality**| City name where the airport is located | Anchor Point|object
**gps_code**| GPS code of the airport | 00AK|object
**iata_code**| IATA code of the airport| | object
**local_code**| Local code airport |00AK | object
**coordinates**|  	GPS coordinates of the airport | -151.695999146, 59.94919968 | object

##### Missing and Duplicate
* no duplicate for the whole Airports dataset
* column `ident` has no missing value and unique.
* none value 45886 in iata_code

In [198]:
df_dup = n_df[n_df.duplicated(keep=False)].sort_values("ident")
df_dup

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates


In [199]:
n_df.nunique()

ident           55075
type                7
name            52144
elevation_ft     5449
continent           6
iso_country       243
iso_region       2810
municipality    27133
gps_code        40850
iata_code        9042
local_code      27436
coordinates     54874
dtype: int64

In [200]:
cols = [col for col in n_df.columns if n_df[col].isnull().any()]
df_miss = n_df[cols]
print(df_miss.isna().sum())
df_miss.head()

elevation_ft     7006
continent       27719
iso_country       247
municipality     5676
gps_code        14045
iata_code       45886
local_code      26389
dtype: int64


,elevation_ft,continent,iso_country,municipality,gps_code,iata_code,local_code
0,11.0,NaN,US,Bensalem,00A,NaN,00A
1,3435.0,NaN,US,Leoti,00AA,NaN,00AA
2,450.0,NaN,US,Anchor Point,00AK,NaN,00AK
3,820.0,NaN,US,Harvest,00AL,NaN,00AL
4,237.0,NaN,US,Newport,NaN,NaN,NaN


In [201]:
df1 = n_df[n_df.apply(lambda x: x.astype(str).str.contains(r'\bUS-HI\b')).any(axis=1)]
df = df1[df1.apply(lambda x: x.astype(str).str.contains('HHW')).any(axis=1)]
df

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates


**c/c : It seems to have no data in common apart from the regions of the united states with the 1st dataset.The columns `ident` contains unique value for airport, digit letter with zero before, sometimes 1 or 2**

### US Cities Demographic Data

* from the US census bureau's
* demographics of all US cities > 65 000


In [121]:
# us-cities-demographics USACITY
nRowsRead = None # change and set to None for the whole data
description   = "us-cities-demographics provide by UDACITY"
name          = "df_demograph"
file          = "us-cities-demographics.csv"

n_df          = pd.read_csv(path+file, sep=";", nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_demograph  = n_df
print(n_df.info())
n_df.head(10)
# dic_5 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}


There are 2891 rows and 12 columns in DataFrame df_demograph.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   City                    2891 non-null   object 
 1   State                   2891 non-null   object 
 2   Median Age              2891 non-null   float64
 3   Male Population         2888 non-null   float64
 4   Female Population       2888 non-null   float64
 5   Total Population        2891 non-null   int64  
 6   Number of Veterans      2878 non-null   float64
 7   Foreign-born            2878 non-null   float64
 8   Average Household Size  2875 non-null   float64
 9   State Code              2891 non-null   object 
 10  Race                    2891 non-null   object 
 11  Count                   2891 non-null   int64  
dtypes: float64(6), int64(2), object(4)
memory usage: 271.2+ KB
None


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
5,Peoria,Illinois,33.1,56229.0,62432.0,118661,6634.0,7517.0,2.40,IL,American Indian and Alaska Native,1343
6,Avondale,Arizona,29.1,38712.0,41971.0,80683,4815.0,8355.0,3.18,AZ,Black or African-American,11592
7,West Covina,California,39.8,51629.0,56860.0,108489,3800.0,37038.0,3.56,CA,Asian,32716
8,O'Fallon,Missouri,36.0,41762.0,43270.0,85032,5783.0,3269.0,2.77,MO,Hispanic or Latino,2583
9,High Point,North Carolina,35.5,51751.0,58077.0,109828,5204.0,16315.0,2.65,NC,Asian,11060


### Data dictionary
Column Name | Description | Example | Type
-|-|-|-|
**City**|City Name|High Point|object
**State**|State Name|North Carolina|object
**Median Age**| The median age of the population | 35.5| float64
**Male Population**| Number of male | 51751.0 | float64
**Female Population** | Number of female | 58077.0 |float64
**Total Population** | Number of toatal population |109828| int64
**Number of Veterans**| Number of veterans |5204.0| float64
**Foreign-born**| Number of residents of the city that were not born (in us)| 16315.0| float64
**Average Household Size**| The average of people in a household| 2.65| float64
**State Code**| The code of the state of the city| NC | object
**Race**|Race class |Asian| object
**Count**|Number of individual of each race|11060|int64

##### missing and duplicate

In [149]:
df_dup = n_df[n_df.duplicated(keep=False)].sort_values("City")
df_dup

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count


In [138]:
cols = [col for col in n_df.columns if n_df[col].isnull().any()]
df_miss = n_df[cols]
df_miss.head()
df_miss[pd.isnull(df_miss).any(axis=1)]


,Male Population,Female Population,Number of Veterans,Foreign-born,Average Household Size
2458,2.93e+04,3.39e+04,4225.0,7.38e+03,2.39
307,2.93e+04,3.39e+04,4225.0,7.38e+03,2.39
1786,2.93e+04,3.39e+04,4225.0,7.38e+03,2.39
567,2.93e+04,3.39e+04,4225.0,7.38e+03,2.39
1648,2.93e+04,3.39e+04,4225.0,7.38e+03,2.39
...,...,...,...,...,...
1826,4.08e+06,4.47e+06,156961.0,3.21e+06,2.68
2494,4.08e+06,4.47e+06,156961.0,3.21e+06,2.68
333,NaN,NaN,15231.0,4.03e+03,NaN
449,NaN,NaN,15231.0,4.03e+03,NaN


In [129]:
duplicateRowsDF = n_df[n_df.duplicated()]
duplicateRowsDF

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count


In [ ]:
# airports_us.csv KAGGLE
nRowsRead = None # change and set to None for the whole data
description = " Dataset from KAGGLE, usefule to follow where go aliens"
name = "df_airport_us"
file = "airports_us.csv"
n_df = pd.read_csv(path+file, nrows = nRowsRead)
nRow, nCol = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, file))
df_airport_us = n_df
df_airport_us.head()
#print(n_df.head(1))
#print(n_df.info())
#dic_1 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_1

In [ ]:
# WDIData.csv Indicators developpement KAGGLE
description      = "WDIData.csv country Indicators developpment KAGGLE"
name             = "df_indicator_dev"
file             = "WDIData.csv"

n_df             = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol       = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_indicator_dev = n_df

dic_3 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
# dic_3

In [ ]:
#I94 DESCRIPTION
description   = "I94_SAS_Labels_Descriptions.SAS"
name          = "df_label_I94"
file          = "I94_SAS_Labels_Descriptions.SAS"

n_df          = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_<>       = n_df

dic_<> = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
dic_<>


In [ ]:
#df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [ ]:
# SAS
description   = " Some parquet files in directory 'sas_data'"
name          = "df_sas"
file          = "./dataset/sas_data"

n_df          = spark.read.parquet("./dataset/sas_data")
nRow, nCol    = n_df.count(), len(n_df.columns)
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_sas        = n_df

dic_8 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_8

In [ ]:
# ./dataset/20-years-us-university-dataset
# decide to not use it for now. 

In [ ]:
# ./dataset/airline-delay-and-cancellation-data-2009-2018/2016.csv
description   = ".Data about us flight in 2016"
name          = "df_airline_delay"
file          = "airline-delay-and-cancellation-data-2009-2018/2016.csv"

n_df          = pd.read_csv(path+file, nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_airline_delay       = n_df

dic_10 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_10
type(dic_10)

In [ ]:
list_dic = [dic_1, dic_2, dic_3, dic_4, dic_5, dic_6, dic_8, dic_9, dic_10]

description   =
name          =
file          =

n_df          = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_temp       = n_df

dic_n = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
dic_n

In [ ]:
# /dataset/education-statistics 

description           = "Data from education-statistics"
name                  = "df_Educ_country_series"
file                  = "education-statistics/EdStatsCountry-Series.csv"
n_df                  = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol            = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_Educ_country_series = n_df
dic_11 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_11

description   = "Data from education-statistics"
name          = "df_Educ_country"
file          = "education-statistics/EdStatsCountry.csv"
n_df          = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_Educ_country = n_df
dic_12 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_12

description   = "Data from education-statistics"
name          = "df_Educ_data"
file          = "education-statistics/EdStatsData.csv"
n_df          = pd.read_csv(path+file, nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_Educ_data  = n_df
dic_13 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_13

description       = "Data from education-statistics"
name              = "df_Educ_foot_note"
file              = "education-statistics/EdStatsFootNote.csv"
n_df              = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol        = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_Educ_foot_note = n_df
dic_14 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_14

description    = "Data from education-statistics"
name           = "df_Educ_series"
file           = "education-statistics/EdStatsSeries.csv"
n_df           = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol     = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_Educ_series = n_df
dic_15 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_15


In [ ]:
list_dic1 = [dic_1, dic_2, dic_3, dic_4, dic_5, dic_6, dic_8, dic_10, dic_11, dic_12, dic_13]
for dic in list_dic1:
    print(type(dic))


In [ ]:
#%xdel n_df
%who_ls dict


----

In [ ]:
#%whos DataFrame
all_df = %who_ls DataFrame
all_df



### df_sas

### df_immigration

### df_indicator_dev

### df_temperature

### df_country

### df_country_series

### df_series

### df_data

### df_foot_note

### df_df_demograph

### df_airport_world

### df_airport_us

### df_airline_delay

In [ ]:
df_airline_delay.head(2)


In [ ]:
df_airline_delay.info()

In [ ]:
#len(df_airline_delay)

In [ ]:
df_airline_delay.columns

In [ ]:
#df_airline_delay.dtypes

In [ ]:
df_airline_delay.describe()

In [ ]:
#df_airline_delay.values

In [ ]:
df_airline_delay.describe(include=np.object)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.